# Coding Excercise: Recommender Systems

**Task**: implement a simple item-based KNN recommendation + content-based mapping algorithm to recommend top-2 items to users (for example, recommend another 2 items for user1). You can use any language. Consider to use proper data structures to make the algorithm efficient for large dataset.

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

W0512 16:42:13.110000 13428 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Instructions for updating:
Colocations handled automatically by placer.


W0512 16:42:25.166000 13428 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\AncillaryRS\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


### Data formatting

Assuming that purchase history of users is represented as item lists of variable length (depending on the number of purchases), normalise the data format and convert it to a long Pandas data frame.<br><br>
User1: item1, item3<br>
User2: item1, item3<br>
User3: item2, item4<br>
...<br>
*Useri: item1, item2, itemj, itemk*<br><br>
**Please let me know if the above assumption is wrong and the data always contains pairs of products (temporally directed graph?)**

In [2]:
input_data = {'User1': ['item1', 'item3'],
             'User2': ['item1', 'item3'],
             'User3': ['item2', 'item4']}

In [3]:
dataset = []

for user in input_data:
    for item in input_data[user]:
        line = [user, item]
        dataset.append(line)

dataset = pd.DataFrame(dataset, columns=['UserID', 'ItemID'])
dataset.head()

,UserID,ItemID
0,User1,item1
1,User1,item3
2,User2,item1
3,User2,item3
4,User3,item2


## Item-Based Collaborative Filtering
### Simple bruteforce approach (suitable if dataset is small)


The purchase data is likely to be sparse. Cosine similarity between two binary vectors can ne defined as 
$$S(_{A,B} = \frac{c}{\sqrt{ab}})$$
where:<br> a = number of occurrences for item A<br>
b = number of occurrences for item B<br>
c = number of shared occurrences for both items (purchased by the same user)

In [4]:
# Calculate number of occurrences for all items
occurrences = dataset.groupby(['ItemID']).count()

# Initialise cache
collab_cache = pd.DataFrame(columns=['Previously purchased item', 'Recommended item', 'Cosine Similarity'])

In [5]:
def cosine_similarity_bin(item):
    """
    Returns cosine similarity between the item and all other items based on user purchasing patterns
    """

    a = occurrences.loc[item][0]
    b = occurrences.values[:,0]
    vector_A = dataset[dataset['ItemID']==item]
    vector_B = dataset
    overlap = pd.merge(vector_A, vector_B, how='inner', on=['UserID']).groupby(['ItemID_y']).count()['UserID']
    c = pd.concat([occurrences, overlap], axis=1, sort=True).fillna(0).iloc[:,1]
    S = c / np.sqrt(a*b)
    
    return pd.concat([pd.Series(np.resize(item, len(S))), pd.Series(S.index), 
                      pd.Series(S.values)], axis=1, sort=False).values

In [6]:
def bruteforce_recs(user, sim_type, repeat_rec=False, caching=True, n=2):
    """
    Given user ID and type of recommendation algorithm required 
    returns averaged item cosine similarities for all possible recommendations
    """
    
    previous_purchases = dataset[dataset['UserID']==user]['ItemID']
    recommendations = []
    
    if sim_type=='semantic':
        unique_items = reviews['ItemID'].unique()
        cos_sim = cosine_similarity_semantic
        cache = review_cache
    elif sim_type=='collaborative':
        unique_items = dataset['ItemID'].unique()
        cos_sim = cosine_similarity_bin
        cache = collab_cache
    else:
        return print('Please use valid sim_type argument: "semantic" or "collaborative"')
     
    for i in previous_purchases:
        rec_retrieved = False
        if (caching == True) & (len(cache) > 0):
            if (i in cache['Previously purchased item'].unique()):
                recommendations.extend(cache[cache['Previously purchased item']==i].values)
                rec_retrieved = True
        if not rec_retrieved:
            S = cos_sim(i)  
            recommendations.extend(S)
            if (caching == True):
                for row in S:
                    cache.loc[len(cache),:] = row
     
    recommendations = pd.DataFrame(recommendations, columns=['Previously purchased item',
                                                                 'Recommended item', 'Cosine Similarity'])
    
    if repeat_rec == False:
        allowed_items = [x for x in unique_items if x not in previous_purchases.values]
        recommendations = recommendations[recommendations['Recommended item'].isin(allowed_items)].reset_index(drop = True)
    
    return recommendations.groupby(['Recommended item']).mean().sort_values(by='Cosine Similarity', 
                                                                            ascending=False).head(n)

Below two best recommendations are returned for *User1* based on simple item-to-item collaborative filtering approach. This is based on the assumption that we don't intend to recommend to repurchase the same items again, as is probably the case with movies, however, this parameter can be easily changed by passing *'repeat_rec=True'* to the function above.<br><br>
In the specific case below, there are only two available items that can be recommended to *User1*, and both of them maximally dissimilar to the purchase history of this user, as there is no purchase history overlap between users in the sample dataset.

In [7]:
bruteforce_recs('User1', 'collaborative')

,Cosine Similarity
Recommended item,
item2,0.0
item4,0.0


### Matrix factorisation (suitable for large datasets)

As alternative to simple KNN search, [Alternating Least Squares (ALS)](http://yifanhu.net/PUB/cf.pdf) (Hu, Koren & Volinsky, 2008; Takács, Pilászy & Tikk, 2011) matrix factorisation model is often used for binary user ratings (e.g., purchase history or click counts) on large datasets.

In [8]:
# Reformat data as a sparse matrix
dense_df = pd.DataFrame()
for col in dataset.columns:
    dense_df[col] = dataset[col].astype("category")
    dense_df[col+'_code'] = dense_df[col].cat.codes
dense_df['Counts'] = 1 # assumin user-item value pairs are unique in the initial dataset

sparse_item_user = sparse.csr_matrix((dense_df['Counts'], (dense_df['ItemID_code'], dense_df['UserID_code'])))
sparse_user_item = sparse.csr_matrix((dense_df['Counts'], (dense_df['UserID_code'], dense_df['ItemID_code'])))
sparse_item_user.todense()

matrix([[1, 1, 0],
        [0, 0, 1],
        [1, 1, 0],
        [0, 0, 1]], dtype=int64)

In [9]:
# Fit the ALS model using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=5, regularization=0.1, iterations=10) # hyperparameters to tune
model.fit(sparse_item_user)

W0512 16:46:56.225999 13428 utils.py:29] Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
100%|████████████████████████████████████████| 10.0/10 [00:00<00:00, 20.96it/s]


In [10]:
def als_recs(user, repeat_rec=False, n=2):
    """
    Given user ID, uses Alternating Least Squares model and returns averaged item similarities 
    for all possible recommendations
    """
    
    user_code = dense_df['UserID'][dense_df['UserID']==user].cat.codes[0]
    recs = model.recommend(user_code, sparse_user_item,  filter_already_liked_items=not repeat_rec, N=n)
    recs = pd.DataFrame(recs, columns=['ItemID_code', 'Approximate Similarity'])
    recs = pd.merge(dense_df[['ItemID', 'ItemID_code']], recs, how='inner', on=['ItemID_code'])
    recs = recs.drop(['ItemID_code'], axis=1)

    return recs.groupby(['ItemID']).mean().sort_values(by='Approximate Similarity', 
                                                                          ascending=False).head(n)

Below two best new recommendations are returned for *User1* based on Alternating Least Squares matrix factorisation approach. <br><br>
In the specific case below, there are only two available items that can be recommended to *User1*. Exact cosine similarities between these items and purchase history of *User1* are equal to 0. Matrix factorisation approximates this by yielding very low similarity scores. 

In [11]:
als_recs('User1')

,Approximate Similarity
ItemID,
item4,-0.001858
item2,-0.001858


## Content-Based Recommendations
Content-based recommender systems typically require a certain degree of insight into the data to gauge which content features are important. With just a handful of examples available, it is hard to evaluate this. Therefore, a generic approach to extracting semantics using a pre-trained model called [Universal Sentence Encoder](https://arxiv.org/pdf/1803.11175.pdf) (Cer at al., 2018) was chosen. The model uses Deep Averaging Network (DAN) model architecture to transform sentences and paragraphs into embedding vectors.<br><br>
In addition to that, sentiment is extracted and used as a proxy for overall item quality. This is done using a simple rule-based general purpose sentiment analysis model called [VADER](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf) (Hutto & Gilbert, 2014). This approach is based on the assumtion that, despite the fact that reviews are subjective and user tastes may differ, if users are generally happy about the product, corresponding recommendation is more likely to be successful.

### Data formatting
For the purpose of this excercise, it is assumed that review authors are unknown, but multiple reviews can be left for the same item. The data is converted to a Pandas dataframe.<br><br>
Item 1:  this movie is really funny. It’s about war.<br>
Item 2:  very sad! I hate it.<br>
Item 3:  very bad movie. British is a nice place to live.<br>
Item 4:  I have no idea about what this director want to say! Peace and war.

In [12]:
review_data = [['item1', 'this movie is really funny. It’s about war.'],
              ['item2', 'very sad! I hate it.'],
              ['item3', 'very bad movie. British is a nice place to live.'],
              ['item4', 'I have no idea about what this director want to say! Peace and war.']]

In [13]:
reviews = pd.DataFrame(review_data, columns=['ItemID', 'Review'])
reviews.head()

,ItemID,Review
0,item1,this movie is really funny. It’s about war.
1,item2,very sad! I hate it.
2,item3,very bad movie. British is a nice place to live.
3,item4,I have no idea about what this director want t...


### Semantics-based recommendations

In [14]:
def get_embeddings(text):
    """
    Given a vector of texts, returns an array of embeddings
    """
    
    text = text.values.tolist()
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embeddings = session.run(embed(text))
    return embeddings

In [15]:
# get embeddings for all reviews
embeddings = get_embeddings(reviews['Review'])
emb_reviews = pd.concat([reviews, pd.DataFrame(embeddings)], axis=1)

# calculate average embedding per item (if multiple reviews are available)
emb_items = emb_reviews.groupby(['ItemID']).mean()

# initialise cache
review_cache = pd.DataFrame(columns=['Previously purchased item', 'Recommended item', 'Cosine Similarity'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0512 16:52:58.798000 13428 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [16]:
def cosine_similarity_semantic(item):
    """
    Returns cosine similarity between the item and all other items based on the review semantics
    """
    S = np.dot(emb_items, emb_items.loc[item])
    #return S
    return pd.concat([pd.Series(np.resize(item, len(S))), pd.Series(emb_items.index), 
                      pd.Series(S)], axis=1, sort=False).values

Below two best recommendations are returned for *User1* based on semantic content-based filtering approach. This is based on the assumption that we don't intend to recommend to repurchase the same items again.<br><br>
In the specific case below, there are only two available items that can be recommended to *User1*. In contrast to collaborative filtering, here we can see that out of the two available movies, the one featuring the word 'war' in the review is ranked higher, as *User1* has already watched a war themed movie before.

In [17]:
bruteforce_recs('User1', sim_type='semantic')

,Cosine Similarity
Recommended item,
item4,0.285893
item2,0.202638


### Sentiment-based recommendations

In [18]:
def get_sentiment(text):
    """
    Given a vector of texts, returns a vector of compound sentiment scores
    """
    
    text = text.values.tolist()
    snts = []
    for t in text:
        snt = analyser.polarity_scores(t)['compound']
        snts.append(snt)
    return snts

In [19]:
# get sentiment for all reviews
sentiment = get_sentiment(reviews['Review'])
sent_reviews = pd.concat([reviews, pd.DataFrame(sentiment, columns=['Sentiment'])], axis=1)

# calculate average sentiment per item (if multiple reviews are available)
sent_items = sent_reviews.groupby(['ItemID']).mean().sort_values(by='Sentiment', ascending=False)

In [20]:
def sentiment_recs(user=None, repeat_rec=False, n=2):
    """
    If non-repeat recomendations are required, returns top n items that have not been previously purchased by user.
    Otherwise, returns top n recommendations by sentiment.
    """

    unique_items = reviews['ItemID'].unique()
    if repeat_rec == False:
        previous_purchases = dataset[dataset['UserID']==user]['ItemID']
        allowed_items = [x for x in unique_items if x not in previous_purchases.values]
    else:
        allowed_items = unique_items

    return sent_items[sent_items.index.isin(allowed_items)].head(n)

Below two new items with the most positive sentiment are returned for *User1*. This is based on the assumption that we don't intend to recommend to repurchase the same items again.

In [21]:
sentiment_recs('User1')

,Sentiment
ItemID,
item4,-0.3802
item2,-0.8254


## Hybrid Recommender System
Based on the performance of separate model components in different scenarios, a number of hybrid algorithms can be applied, such as:
* weighted approach, where scores from each component are combined using a linear function
* mixed approach, where top *n* recommendations from each component are combined into a single final list
* switching approach, where certain component is preferred based on a specific scenario (e.g., movies with the highest sentiment are recommended to brand-new users with no purchase history, while collaborative filtering is used for everyone else)
* enseble model, where best way of combining components determined by a machine learning approach<br>

Here, due to limited amount of data available, a simple weighted approach with equal weights will be used. 

In [22]:
def hybrid_recs(user, repeat_rec=False, weights=[1, 1, 1], n=2):
    """
    Given user ID and weights returns the result of a linear function of individual recommender components.
    """
    
    collaborative = bruteforce_recs(user, 'collaborative', repeat_rec, n=10)
    semantic = bruteforce_recs(user, 'semantic', repeat_rec, n=10)
    sentiment = sentiment_recs(user, repeat_rec, n=10)
    
    all_recs = pd.concat([collaborative, semantic, sentiment], axis=1, sort=True)
    all_recs.columns = ['Collaborative Filtering', 'Content Filtering', 'Sentiment']
    all_recs['Hybrid'] = weights[0] * all_recs['Collaborative Filtering'].values + \
                            weights[1] * all_recs['Content Filtering'].values + \
                            weights[2] * all_recs['Sentiment'].values
    
    return all_recs.sort_values(by='Hybrid', ascending=False).head(n)

Below two new items with the highest combined score are returned for *User1*. This is based on the assumption that we don't intend to recommend to repurchase the same items again.<br><br>
The combined recommendations allow to utilise all sources of information in ranking the recommendations. While none of the recommended items have been purchased by anyone who has a similar purchasing history as *User1*, the review for *item4* is semantically more similar to the reviews for items previously purchased by this user, and it is also more positive. Taken together, out of the two possible recommendations, *item4* is ranked higher than *item2*.<br><br>
In real-world scenario it is unhighly likely that individual components have equal relevance, so the weighting should be tuned to reflect that.

In [23]:
hybrid_recs('User1')

,Collaborative Filtering,Content Filtering,Sentiment,Hybrid
item4,0.0,0.285893,-0.3802,-0.094307
item2,0.0,0.202638,-0.8254,-0.622762


If repeat recommendations are allowed, we can see that sentiment component allows to differentiate between otherwise equivalent items - a movie with more positive overall review is ranked higher. Please note that repeat recommendations have not been penalised here in any way, so previous purchases are implicitly pushed to the top because of the high cosine similarity scores.

In [24]:
hybrid_recs('User1', repeat_rec=True, n=4)

,Collaborative Filtering,Content Filtering,Sentiment,Hybrid
item1,1.0,0.769838,-0.1796,1.590238
item3,1.0,0.769837,-0.2484,1.521437
item4,0.0,0.285893,-0.3802,-0.094307
item2,0.0,0.202638,-0.8254,-0.622762


## Possible Next Steps

* Improve individual components - e.g., use more advanced sentiment model to extract sentiment that is related to the quality of the item (consider using the sentiment from the first, but not the second part of this review: *"very bad movie. British is a nice place to live."*)
* Improve efficiency of the algorithms by using approximate KNN search functions.
* Build an ensemble model to combine all sources of data in the most optimal way.
* Select the most optimal models and tune hyperparameters based on the model performance metrics.
* Use an algorithm that extracts meaningful semantic features from reviews rather than using untransformed generic embeddings. E.g., reframe the algorithms as a supervised learning task to optimise the way information is extracted from reviews.
* Combine review semantics and sentiment in a meaningful way - e.g., use negative sentiment as an indication that semantics used in a review are negatively assicoated with user's tastes.